In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
import re
import uuid
from llama_index.core.schema import MetadataMode
from tqdm.notebook import tqdm
import torch
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings
from llama_index.core.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
)
from llama_index.experimental.query_engine.pandas import (
    PandasInstructionParser,
)
from llama_index.core import PromptTemplate
from IPython.display import Markdown, display

In [3]:
# Model names (make sure you have access on HF)
LLAMA2_7B = "meta-llama/Llama-2-7b-hf"
LLAMA2_7B_CHAT = "meta-llama/Llama-2-7b-chat-hf"
LLAMA2_13B = "meta-llama/Llama-2-13b-hf"
LLAMA2_13B_CHAT = "meta-llama/Llama-2-13b-chat-hf"
LLAMA2_70B = "meta-llama/Llama-2-70b-hf"
LLAMA2_70B_CHAT = "meta-llama/Llama-2-70b-chat-hf"

LLAMA3_70B_INST = "meta-llama/Meta-Llama-3-70B-Instruct"
LLAMA3_70B = "meta-llama/Meta-Llama-3-70B"
CodeLlama_70b_py_hf = "meta-llama/CodeLlama-70b-Python-hf"
CodeLlama_13b_py_hf = "meta-llama/CodeLlama-13b-Python-hf"

selected_model = LLAMA2_70B_CHAT

# SYSTEM_PROMPT = """\
#     Context information is below.
    
#     ---------------------
    
#     Given the context information and not prior knowledge.
#     generate only questions based on the below query.
    
#     You are a Teacher/ Professor. Your task is to setup \
#     questions for an upcoming \
#     quiz/examination. The questions should be diverse in nature \
#     across the document. Restrict the questions to the \
#     context information provided."
#     """

SYSTEM_PROMPT = """\
    Context information is below.
    
    ---------------------
    """

query_wrapper_prompt = PromptTemplate(
    "[INST]<>\n" + SYSTEM_PROMPT + "<>\n\n{query_str}[/INST] "
)

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=2048,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name=selected_model,
    model_name=selected_model,
    device_map="auto",
    # change these settings below depending on your GPU
    model_kwargs={"torch_dtype": torch.float16, "load_in_8bit": True},
)


/home/hb/myenv/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

/home/hb/myenv/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


# BGP data read

In [5]:
import pandas as pd
# df = pd.read_json("/home/hb/dataset_bgp/bgp_nlp_dataset/google_leak_main_with_anomalies.json")

df= pd.read_csv("/home/hb/BGP_data_analysis/pybgpstream/bgp_features_asn_15169.csv")
df

,timestamp,asn,num_routes,num_new_routes,num_withdrawals,num_origin_changes,num_route_changes,max_path_length,avg_path_length,max_edit_distance,avg_edit_distance,num_announcements,num_unique_prefixes_announced
0,2018-04-25 08:30:00,15169,0,0,0,0,0,0,0.000000,0,0.000000,0,0
1,2018-04-25 08:35:00,15169,0,0,0,0,0,0,0.000000,0,0.000000,0,0
2,2018-04-25 08:40:00,15169,0,0,0,0,0,0,0.000000,0,0.000000,0,0
3,2018-04-25 08:45:00,15169,0,0,0,0,0,0,0.000000,0,0.000000,0,0
4,2018-04-25 08:50:00,15169,0,0,0,0,0,0,0.000000,0,0.000000,0,0
5,2018-04-25 08:55:00,15169,0,0,0,0,0,0,0.000000,0,0.000000,0,0
6,2018-04-25 09:00:00,15169,0,0,0,0,0,0,0.000000,0,0.000000,0,0
7,2018-04-25 09:05:00,15169,0,0,0,0,0,0,0.000000,0,0.000000,0,0
8,2018-04-25 09:10:00,15169,0,0,0,0,0,0,0.000000,0,0.000000,0,0
9,2018-04-25 09:15:00,15169,0,0,0,0,0,0,0.000000,0,0.000000,0,0


In [15]:
instruction_str = (
    "1. Convert the query to executable Python code using Pandas.\n"
    "2. The final line of code should be a Python expression that can be called with the `eval()` function.\n"
    "3. The code should represent a solution to the query.\n"
    "4. PRINT ONLY THE EXPRESSION.\n"
    "5. Do not quote the expression.\n"
)

pandas_prompt_str = (
    "You are working with a pandas dataframe in Python.\n"
    "The name of the dataframe is `df`.\n"
    "This is the result of `print(df.head())`:\n"
    "{df_str}\n\n"
    "Follow these instructions:\n"
    "{instruction_str}\n"
    "Query: {query_str}\n\n"
    "Expression:"
)
response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results.\n"
    "Query: {query_str}\n\n"
    "Pandas Instructions (optional):\n{pandas_instructions}\n\n"
    "Pandas Output: {pandas_output}\n\n"
    "Response: "
)

pandas_prompt = PromptTemplate(pandas_prompt_str).partial_format(
    instruction_str=instruction_str, df_str=df.head(5)
)
pandas_output_parser = PandasInstructionParser(df)
response_synthesis_prompt = PromptTemplate(response_synthesis_prompt_str)

In [16]:
qp = QP(
    modules={
        "input": InputComponent(),
        "pandas_prompt": pandas_prompt,
        "llm1": llm,
        "pandas_output_parser": pandas_output_parser,
        "response_synthesis_prompt": response_synthesis_prompt,
        "llm2": llm,
    },
    verbose=True,
)
qp.add_chain(["input", "pandas_prompt", "llm1", "pandas_output_parser"])
qp.add_links(
    [
        Link("input", "response_synthesis_prompt", dest_key="query_str"),
        Link(
            "llm1", "response_synthesis_prompt", dest_key="pandas_instructions"
        ),
        Link(
            "pandas_output_parser",
            "response_synthesis_prompt",
            dest_key="pandas_output",
        ),
    ]
)
# add link from response synthesis prompt to llm2
qp.add_link("response_synthesis_prompt", "llm2")

In [20]:
response = qp.run(
    query_str="analyze the bgp dataset. output the timestamp and determine the type of anomaly.",
)

> Running module input with input: 
query_str: analyze the bgp dataset. output the timestamp and determine the type of anomaly.

> Running module pandas_prompt with input: 
query_str: analyze the bgp dataset. output the timestamp and determine the type of anomaly.

> Running module llm1 with input: 
prompt: You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
This is the result of `print(df.head())`:
             timestamp    asn  num_routes  num_new_routes  num_withdrawa...



/home/hb/myenv/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/hb/myenv/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Python version must be >= 3.9 in order to use the default output processor, which executes the Python query. Instead, we will return the raw Python instructions as a string.


> Running module pandas_output_parser with input: 
input: 
```
import pandas as pd

df = pd.read_csv('bgp.csv')

# filter rows where num_routes, num_new_routes, num_withdrawals are 0
filtered_df = df[df['num_routes'] > 0 | df['num_new_routes'] > 0 | df['num_...

> Running module response_synthesis_prompt with input: 
query_str: analyze the bgp dataset. output the timestamp and determine the type of anomaly.
pandas_instructions: 
```
import pandas as pd

df = pd.read_csv('bgp.csv')

# filter rows where num_routes, num_new_routes, num_withdrawals are 0
filtered_df = df[df['num_routes'] > 0 | df['num_new_routes'] > 0 | df['num_...
pandas_output: 
```
import pandas as pd

df = pd.read_csv('bgp.csv')

# filter rows where num_routes, num_new_routes, num_withdrawals are 0
filtered_df = df[df['num_routes'] > 0 | df['num_new_routes'] > 0 | df['num_...

> Running module llm2 with input: 
prompt: Given an input question, synthesize a response from the query results.
Query: analyze the bgp dataset

In [21]:
# LLama2_70B
display(Markdown(f"{response}"))


The output of the code will be a list of tuples, where each tuple contains a timestamp and an anomaly score. The timestamp is in the format 'YYYY-MM-DD HH:MM:SS', and the anomaly score is a floating-point number representing the difference between the count of unique prefixes announced for that timestamp and the mean of the count of unique prefixes announced for all timestamps.

For example, the output could be:

[('2022-01-01 00:00:00', 0.0), ('2022-01-01 00:01:00', 0.2), ('2022-01-01 00:02:00', -0.3), ...]

The anomaly score can be used to identify anomalies in the BGP updates. A positive anomaly score indicates that the count of unique prefixes announced for that timestamp is higher than the mean, which could indicate an anomaly. A negative anomaly score indicates that the count of unique prefixes announced for that timestamp is lower than the mean, which could also indicate an anomaly. The threshold for determining whether a score is an anomaly can be set based on the specific requirements of the problem.

## LLaMA2-70B answers:


1. The given BGP dataset contains several anomalies. The rows with anomalies and their corresponding timestamps and anomaly types are as follows:

2018-04-25 08:40:00 - Anomaly: Large number of routes (num_routes = 1050)
2018-04-25 08:45:00 - Anomaly: Large number of withdrawals (num_withdrawals = 120)
2018-04-25 08:50:00 - Anomaly: Large number of origin changes (num_origin_changes = 7)
These anomalies may indicate issues with the BGP configuration or network infrastructure. It is recommended to investigate these anomalies further to determine their root cause and take appropriate action to resolve them.

2. The given code uses the Pandas library to detect anomalies in a dataset. 

It calculates the moving average of the 'nb_A' and 'nb_W' columns with a window size of 3, and then calculates the difference between the current value and the moving average. It then calculates the absolute difference between the current value and the moving average, and repeats the same process for the 'nb_implicit_W' column. Finally, it creates a new column called 'anomaly_status' and assigns it a value of 'no anomalies detected' for all rows, and then replaces the values in this column with 'anomaly detected' for any rows where the absolute difference between the current value and the moving average is greater than 2.
The code seems to be well-structured and easy to follow. However, there is an error in the output, which suggests that there is a problem with the code. The error message indicates that there is an unexpected EOF (end-of-file) while parsing the code. This could be due to a syntax error or a missing closing bracket.
To fix the error, we need to carefully review the code and ensure that all brackets and parentheses are properly closed. Additionally, we should check for any syntax errors or typos that could be causing the error. Once we have fixed the error, we should be able to run the code successfully and generate the output.
In summary, the given code appears to be well-structured and easy to follow, but there is an error in the output that needs to be fixed before we can run the code successfully.